Paso 1: Descarga de mseed por estacion que contienen solo 1 sismo y sus canales

In [ ]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import os

csv_file = 'Consultas_SGC/_Catalogo_Ultimo.csv'

# Leer CSV y limpiar encabezados
df = pd.read_csv(csv_file, skipinitialspace=True)
df.columns = df.columns.str.strip().str.replace('"', '')

print("Columnas encontradas:", df.columns.tolist())

# Sin el límite de descargas
count = 0

download_folder = 'A_REDNE_mseed_downloads'
os.makedirs(download_folder, exist_ok=True)

# Estaciones de interés: UIS01, UIS02, ..., UIS11
stations = [f"UIS{str(i).zfill(2)}" for i in range(1, 12)]

for idx, row in df.iterrows():
    try:
        # Ajusta el nombre exacto de la columna después de limpiar
        start_time_str = row['Fecha-Hora  (UTC)']

        # Restar minutos para start_time y sumar minutos para end_time
        base_time = datetime.strptime(start_time_str, '%Y-%m-%d %H:%M:%S')
        start_time = base_time - timedelta(minutes=0.1)
        end_time = base_time + timedelta(minutes=1.9)

        start_fmt = start_time.strftime('%Y-%m-%dT%H:%M:%S')
        end_fmt = end_time.strftime('%Y-%m-%dT%H:%M:%S')

        base_url = "https://osso.univalle.edu.co/fdsnws/dataselect/1/query"

        # Iterar sobre cada estación
        for station in stations:
            url = (f"{base_url}?starttime={start_fmt}&endtime={end_fmt}"
                   f"&network=UX&station={station}&channel=***&nodata=404")

            #print(f"\n[{count+1}] Intentando descargar archivo para estación {station} y rango: {start_fmt} - {end_fmt}")
            #print(f"URL: {url}")

            # Desactivar la verificación de certificado SSL
            response = requests.get(url, timeout=15, verify=False)

            if response.status_code != 200:
                #print(f"    Error HTTP {response.status_code} - saltando estación {station}...")
                continue

            if b'no waveform data found' in response.content.lower():
                #print(f"    No hay datos para la estación {station} en este rango, saltando...")
                continue

            filename = f"{station}_{start_fmt.replace(':', '-')}_{end_fmt.replace(':', '-')}.mseed"
            filepath = os.path.join(download_folder, filename)

            with open(filepath, 'wb') as f:
                f.write(response.content)

            print(f"    Archivo guardado en: {filepath}")
            count += 1

    except Exception as e:
        #print(f"    Error inesperado en fila {idx}: {e}, saltando...")
        continue

print(f"Descargas completadas: {count} archivos descargados.")


Paso 2: La estacion UIS05, se separa en 2, una real con sus canales de aceleracion, y otra estacion virtual con canales de velocidad, como se ve la estacion UIS05 tiene 4 canales EHZ, ENZ, ENE y ENN, 3 de aceleracion (ENZ, ENE y ENN) y  1 de velocidad (EHZ), lo que quiero es que  para la estacion virtual se me duplique el canal (EHZ) en 2 canales mas y que se tranforemen en (EHE) ESTE y (EHN) NORTE y se llame UIS05V, y la otra estacion UIS05 queda solo con los 3 canales de aceleracion (ENZ, ENE y ENN)

In [ ]:
import os
from obspy import read
import shutil

# Definir la carpeta donde están los archivos mseed descargados
download_folder = 'A_REDNE_mseed_downloads'
output_folder = 'B_REDNE_mseed_UIS05virtual'
os.makedirs(output_folder, exist_ok=True)

# Función para procesar cada archivo mseed y separarlo
def process_mseed(file_path):
    # Leer el archivo mseed
    st = read(file_path)

    # Comprobar si la estación en el archivo es UIS05
    is_uis05 = any('UIS05' in tr.stats.station for tr in st)

    # Si no es de la estación UIS05, copiar el archivo tal cual está
    if not is_uis05:
        shutil.copy(file_path, output_folder)
        print(f"Archivo {file_path} copiado sin cambios.")
        return

    # Crear dos streams separados para los canales de aceleración y velocidad
    acceleration_stream = []
    velocity_stream = []

    # Filtrar los canales de la estación UIS05
    for tr in st:
        if 'UIS05' in tr.stats.station:
            if tr.stats.channel in ['ENZ', 'ENE', 'ENN']:
                # Agregar canales de aceleración al stream de aceleración
                acceleration_stream.append(tr)
            elif tr.stats.channel == 'EHZ':
                # Agregar canal de velocidad EHZ al stream de velocidad
                velocity_stream.append(tr)
                
                # Crear los canales EHE y EHN con los mismos datos que EHZ
                tr_ehe = tr.copy()
                tr_ehe.stats.channel = 'EHE'
                tr_ehn = tr.copy()
                tr_ehn.stats.channel = 'EHN'
                
                # Agregar los nuevos canales al stream de velocidad
                velocity_stream.append(tr_ehe)
                velocity_stream.append(tr_ehn)

    # Si tenemos datos en los streams de aceleración y velocidad, los guardamos en archivos separados
    if len(acceleration_stream) > 0:
        # Guardar el archivo de aceleración
        acceleration_stream = st.__class__(acceleration_stream)  # Convertir lista en Stream
        acceleration_file = os.path.join(output_folder, f"UIS05_acceleration_{os.path.basename(file_path)}")
        acceleration_stream.write(acceleration_file, format='MSEED')
        print(f"Archivo de aceleración guardado: {acceleration_file}")
    
    if len(velocity_stream) > 0:
        # Guardar el archivo de velocidad
        velocity_stream = st.__class__(velocity_stream)  # Convertir lista en Stream
        velocity_file = os.path.join(output_folder, f"UIS05_velocity_{os.path.basename(file_path)}")
        velocity_stream.write(velocity_file, format='MSEED')
        print(f"Archivo de velocidad guardado: {velocity_file}")

# Iterar sobre todos los archivos mseed en la carpeta de descargas
for filename in os.listdir(download_folder):
    if filename.endswith('.mseed'):
        file_path = os.path.join(download_folder, filename)
        process_mseed(file_path)

print("Procesamiento completado.")




Paso 3: eliminacion de mseed con menos de 6000 muestras

In [ ]:
from obspy import read
import os

# Carpeta que contiene los archivos mseed
input_folder = 'B_REDNE_mseed_UIS05virtual'
valid_folder = 'C_Valid_mseed_files'  # Carpeta para los archivos válidos

# Crear la carpeta para archivos válidos si no existe
os.makedirs(valid_folder, exist_ok=True)

# Iterar sobre todos los archivos mseed en la carpeta
for filename in os.listdir(input_folder):
    if filename.endswith('.mseed'):
        file_path = os.path.join(input_folder, filename)

        try:
            # Leer el archivo mseed
            st = read(file_path)

            # Verificar si algún canal tiene menos de 6000 muestras
            discard_file = False
            for tr in st:
                if len(tr.data) < 6000:
                    print(f"Descartando archivo {filename}: el canal {tr.stats.channel} tiene menos de 6000 muestras.")
                    discard_file = True
                    break  # Si encontramos un canal que no cumple, descartamos el archivo

            # Si el archivo no se descarta, moverlo a la carpeta de archivos válidos
            if not discard_file:
                print(f"Archivo {filename} aprobado, tiene canales con al menos 6000 muestras.")
                valid_file_path = os.path.join(valid_folder, filename)
                os.rename(file_path, valid_file_path)
                print(f"Archivo {filename} movido a la carpeta de archivos válidos.")
            else:
                # Si el archivo no cumple, lo eliminamos directamente
                os.remove(file_path)
                print(f"Archivo {filename} eliminado debido a que no cumple con el requisito de 6000 muestras.")

        except Exception as e:
            print(f"Error al procesar el archivo {filename}: {e}")


Paso 4: Eliminar tendencia y media según STEAD

In [ ]:
from obspy import read
import os

# Definir la carpeta donde están los archivos mseed
download_folder = 'C_Valid_mseed_files'
output_folder = 'D_REDNE_mseed_tendencia_media'
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos mseed en la carpeta de descargas
for filename in os.listdir(download_folder):
    if filename.endswith('.mseed'):
        file_path = os.path.join(download_folder, filename)

        # Leer el archivo mseed
        st = read(file_path)
        
        # Iterar sobre todas las trazas (canales) en el archivo
        for tr in st:
            # Eliminar la tendencia (ajuste de una línea recta)
            tr.detrend(type='linear')  # Ajuste de tendencia lineal
            print(f"Tendencia eliminada en canal {tr.stats.channel} del archivo {filename}")

            # Eliminar la media
            tr.detrend(type='constant')  # Eliminar la media
            print(f"Media eliminada en canal {tr.stats.channel} del archivo {filename}")

        # Guardar el archivo procesado
        processed_filename = os.path.join(output_folder, f"processed_{filename}")
        st.write(processed_filename, format='MSEED')
        print(f"Archivo procesado guardado: {processed_filename}")

print("Procesamiento completado.")


Adaptado y aplicando STA/LTA a los datos mseed para realizar el recorte de la traza y que inicien al mismo tiempo los canales

In [ ]:
"""
Procesamiento en lote de detección de eventos STA/LTA para archivos MSEED.

-------
* AL leer cada archivo, si el MSEED termina con exactamente 2 eventos,
  se recorta la señal completa (todas las trazas del archivo) a una ventana de
  1 minuto que comienza 5s antes del primer evento detectado.
* Los archivos recortados se guardan en la carpeta `OUTPUT_DIR = mseed_dos_eventos_recortados`

FLUJO
--------------
1. Analiza solo la primera traza para la detección inicial de eventos.
2. Aplica STA/LTA con `TH_ON = 3.5` y reprocesa con umbrales 2.7 ó 6.0 según el número de eventos (igual que antes).
3. Cuenta archivos con 0, 1, 2, 3 y ≥4 eventos y lista nombres de los grupos 0, 3 y ≥4.
4. Recorta los archivos con 2 eventos y los guarda aparte.
"""

import os
import numpy as np
from datetime import timedelta
from pathlib import Path

from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta_py, trigger_onset

# --------------------------- PARÁMETROS ---------------------------------- #
DATA_DIR = "D_REDNE_mseed_tendencia_media"      # Carpeta con los .mseed
OUTPUT_DIR = "E_mseed_eventos_umbrales_finales"      # Carpeta de salida recortes
STA_SECONDS = 1
LTA_SECONDS = 30        
TH_OFF = 1.6            
TH_ON_DEFAULT = 4       
TH_ON_LOW = 3        
TH_ON_HIGH = 6          
# ------------------------------------------------------------------------- #

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

def detectar_eventos(traza, th_on):
    """Devuelve lista de tiempos UTC (datetime) donde se detectan eventos."""
    n_sta = int(STA_SECONDS * traza.stats.sampling_rate)
    n_lta = int(LTA_SECONDS * traza.stats.sampling_rate)
    cft = classic_sta_lta_py(traza.data, n_sta, n_lta)
    on_off = trigger_onset(cft, th_on, TH_OFF)
    base_time = traza.stats.starttime.datetime
    return [base_time + timedelta(seconds=traza.times()[idx_on]) for idx_on, _ in on_off]

def procesar_archivo(path, th_on):
    """Devuelve (canal_id, lista_eventos) usando solo la primera traza."""
    st = read(path)
    tr = st[0]
    eventos = detectar_eventos(tr, th_on)
    return tr.id, eventos


def recortar_y_guardar(path, primer_evento_dt):
    """Recorta todas las trazas a 60 s iniciando 5 s antes de *primer_evento_dt*."""
    st = read(path)
    start = UTCDateTime(primer_evento_dt) - 5
    # Truncar milisegundos y microsegundos dejando solo los segundos exactos
    start = UTCDateTime(int(start.timestamp))
    end = start + 60
    st_rec = st.copy().trim(starttime=start, endtime=end, pad=True, fill_value=0)
    out_path = os.path.join(OUTPUT_DIR, os.path.basename(path))
    st_rec.write(out_path, format="MSEED")


def main():
    archivos = sorted(f for f in os.listdir(DATA_DIR) if f.lower().endswith(".mseed"))

    archivos_0 = []
    archivos_1 = []
    archivos_2 = [] 
    archivos_3 = []
    archivos_4mas = []

    for fname in archivos:
        path = os.path.join(DATA_DIR, fname)

        try:
            canal_id, eventos = procesar_archivo(path, TH_ON_DEFAULT)
        except Exception as exc:
            print(f"[ERROR] {fname}: {exc}")
            continue

        total_inicial = len(eventos)

        # Reprocesar según la cantidad inicial de eventos
        if total_inicial == 1:
            _, eventos = procesar_archivo(path, TH_ON_LOW)

        elif total_inicial >= 3:
            _, eventos = procesar_archivo(path, TH_ON_HIGH)

        total_final = len(eventos)

        # Actualizar listas y contadores
        if total_final == 0:
            archivos_0.append(fname)
        elif total_final == 1:
            archivos_1.append(fname)
        elif total_final == 2:
            archivos_2.append((fname, eventos)) # se guardan los eventos y se recorta
        elif total_final == 3:
            archivos_3.append(fname)
        else:  # ≥4
            archivos_4mas.append(fname)

    # Se recortan archivos con 2 eventos
    for fname, eventos in archivos_2:
        path = os.path.join(DATA_DIR, fname)
        primer_evt = min(eventos)
        recortar_y_guardar(path, primer_evt)

    # Resumen global
    print("\n============= RESUMEN GLOBAL =============")
    print(f"Total de archivos analizados : {len(archivos)}")
    print(f"Archivos con 0 eventos       : {len(archivos_0)}")
    print(f"Archivos con 1 evento        : {len(archivos_1)}")
    print(f"Archivos con 2 eventos       : {len(archivos_2)}")
    print(f"Archivos con 3 eventos       : {len(archivos_3)}")
    print(f"Archivos con ≥4 eventos      : {len(archivos_4mas)}")

    print("\n→ Archivos con 0 eventos:")
    for f in archivos_0:
        print(f"   - {f}")

    print("\n→ Archivos con 1 eventos:")
    for f in archivos_1:
        print(f"   - {f}")

    print("\n→ Archivos con 3 eventos:")
    for f in archivos_3:
        print(f"   - {f}")

    print("\n→ Archivos con 4 o más eventos:")
    for f in archivos_4mas:
        print(f"   - {f}")


if __name__ == "__main__":
    main()


In [1]:
"""
Procesamiento en lote de detección de eventos STA/LTA para archivos MSEED.

-------
* AL leer cada archivo, si el MSEED termina con exactamente 2 eventos,
  se recorta la señal completa (todas las trazas del archivo) a una ventana de
  1 minuto que comienza 5s antes del primer evento detectado.
* Los archivos recortados se guardan en la carpeta `OUTPUT_DIR = mseed_dos_eventos_recortados`

FLUJO
--------------
1. Analiza solo la primera traza para la detección inicial de eventos.
2. Aplica STA/LTA con `TH_ON = 3.5` y reprocesa con umbrales 2.7 ó 6.0 según el número de eventos (igual que antes).
3. Cuenta archivos con 0, 1, 2, 3 y ≥4 eventos y lista nombres de los grupos 0, 3 y ≥4.
4. Recorta los archivos con 2 eventos y los guarda aparte.
"""

import os
import numpy as np
from datetime import timedelta
from pathlib import Path

from obspy import read, UTCDateTime
from obspy.signal.trigger import classic_sta_lta_py, trigger_onset

# --------------------------- PARÁMETROS ---------------------------------- #
DATA_DIR = "D_REDNE_mseed_tendencia_media"      # Carpeta con los .mseed
OUTPUT_DIR = "E_mseed_eventos_umbrales_finales1"      # Carpeta de salida recortes
STA_SECONDS = 1
LTA_SECONDS = 30        
TH_OFF = 1.6            
TH_ON_DEFAULT = 4       
TH_ON_LOW = 3        
TH_ON_HIGH = 6          
# ------------------------------------------------------------------------- #

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

def detectar_eventos(traza, th_on):
    """Devuelve lista de tiempos UTC (datetime) donde se detectan eventos."""
    n_sta = int(STA_SECONDS * traza.stats.sampling_rate)
    n_lta = int(LTA_SECONDS * traza.stats.sampling_rate)
    cft = classic_sta_lta_py(traza.data, n_sta, n_lta)
    on_off = trigger_onset(cft, th_on, TH_OFF)
    base_time = traza.stats.starttime.datetime
    return [base_time + timedelta(seconds=traza.times()[idx_on]) for idx_on, _ in on_off]

def procesar_archivo(path, th_on):
    """Devuelve (canal_id, lista_eventos) usando solo la primera traza."""
    st = read(path)
    tr = st[0]
    eventos = detectar_eventos(tr, th_on)
    return tr.id, eventos


def recortar_y_guardar(path, primer_evento_dt):
    """Recorta todas las trazas a 60 s iniciando 5 s antes de *primer_evento_dt*."""
    st = read(path)
    start = UTCDateTime(primer_evento_dt) - 5
    # Truncar milisegundos y microsegundos dejando solo los segundos exactos
    start = UTCDateTime(int(start.timestamp))
    end = start + 60
    st_rec = st.copy().trim(starttime=start, endtime=end, pad=True, fill_value=0)
    out_path = os.path.join(OUTPUT_DIR, os.path.basename(path))
    st_rec.write(out_path, format="MSEED")


def main():
    archivos = sorted(f for f in os.listdir(DATA_DIR) if f.lower().endswith(".mseed"))

    archivos_0 = []
    archivos_1 = []
    archivos_2 = [] 
    archivos_3 = []
    archivos_4mas = []

    for fname in archivos:
        path = os.path.join(DATA_DIR, fname)

        try:
            canal_id, eventos = procesar_archivo(path, TH_ON_DEFAULT)
        except Exception as exc:
            print(f"[ERROR] {fname}: {exc}")
            continue

        total_inicial = len(eventos)

        # Reprocesar según la cantidad inicial de eventos
        if total_inicial == 1:
            _, eventos = procesar_archivo(path, TH_ON_LOW)

        elif total_inicial >= 3:
            _, eventos = procesar_archivo(path, TH_ON_HIGH)

        total_final = len(eventos)

        # Actualizar listas y contadores
        if total_final == 0:
            archivos_0.append(fname)
        elif total_final == 1:
            archivos_1.append(fname)
        elif total_final == 2:
            archivos_2.append((fname, eventos)) # se guardan los eventos y se recorta
        elif total_final == 3:
            archivos_3.append(fname)
        else:  # ≥4
            archivos_4mas.append(fname)

    # Se recortan archivos con 2 eventos
    for fname, eventos in archivos_2:
        path = os.path.join(DATA_DIR, fname)

        # Ordenar eventos por seguridad
        eventos_ordenados = sorted(eventos)
        primer_evt = eventos_ordenados[0]
        segundo_evt = eventos_ordenados[1]

        # Calcular inicio del recorte
        start = UTCDateTime(primer_evt) - 5
        start = UTCDateTime(int(start.timestamp))  # eliminar ms

        # Condición: segundo evento < 30s después del inicio del recorte
        if UTCDateTime(segundo_evt) <= start + 30:
            recortar_y_guardar(path, primer_evt)
        else:
            print(f"[IGNORADO] {fname}: segundo evento muy tarde")


    # Resumen global
    print("\n============= RESUMEN GLOBAL =============")
    print(f"Total de archivos analizados : {len(archivos)}")
    print(f"Archivos con 0 eventos       : {len(archivos_0)}")
    print(f"Archivos con 1 evento        : {len(archivos_1)}")
    print(f"Archivos con 2 eventos       : {len(archivos_2)}")
    print(f"Archivos con 3 eventos       : {len(archivos_3)}")
    print(f"Archivos con ≥4 eventos      : {len(archivos_4mas)}")

    print("\n→ Archivos con 0 eventos:")
    for f in archivos_0:
        print(f"   - {f}")

    print("\n→ Archivos con 1 eventos:")
    for f in archivos_1:
        print(f"   - {f}")

    print("\n→ Archivos con 3 eventos:")
    for f in archivos_3:
        print(f"   - {f}")

    print("\n→ Archivos con 4 o más eventos:")
    for f in archivos_4mas:
        print(f"   - {f}")


if __name__ == "__main__":
    main()


[IGNORADO] processed_UIS01_2024-05-31T18-10-35_2024-05-31T18-12-35.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-06-02T00-27-24_2024-06-02T00-29-24.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-10-03T14-52-06_2024-10-03T14-54-06.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-10-13T16-57-41_2024-10-13T16-59-41.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-03T18-55-13_2024-11-03T18-57-13.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-04T15-09-31_2024-11-04T15-11-31.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-05T15-21-31_2024-11-05T15-23-31.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-10T01-45-11_2024-11-10T01-47-11.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-13T00-19-36_2024-11-13T00-21-36.mseed: segundo evento muy tarde
[IGNORADO] processed_UIS01_2024-11-13T22-08-30_2024-11-13T22-10-30.mseed: segundo evento muy tarde
[IGNORADO]

Alineacion de tiempos y cambio de nombre para su trazabilidad

In [ ]:
"""
Organiza los archivos MSEED recortados (1 min, 3 canales) en una sola carpeta.

- Lee todos los .mseed que encuentre en la carpeta `E_mseed_eventos_umbrales_finales`.
- Guarda cada traza directamente dentro de <OUTPUT_BASE>.
- Nomenclatura de salida:
    <NETWORK>.<STATION>..<CHANNEL>__YYYYMMDDThhmmssZ__YYYYMMDDThhmmssZ.mseed
"""

import os
from pathlib import Path
from obspy import read
from obspy import UTCDateTime

# --------------------------- PARÁMETROS ---------------------------------- #
INPUT_DIR = "E_mseed_eventos_umbrales_finales"  # Carpeta con recortes de 1 min
OUTPUT_BASE = "F_mseed_eventos_finales"      # Carpeta raíz de salida
TIME_FMT = "%Y%m%dT%H%M%SZ"                     # Formato de fecha
# ------------------------------------------------------------------------- #

Path(OUTPUT_BASE).mkdir(parents=True, exist_ok=True)


def main():
    for fname in sorted(p for p in os.listdir(INPUT_DIR) if p.lower().endswith(".mseed")):
        path = os.path.join(INPUT_DIR, fname)
        try:
            st = read(path)
        except Exception as exc:
            print(f"[ERROR] {fname}: {exc}")
            continue

        for tr in st:
            net = tr.stats.network or "UX"
            sta = tr.stats.station
            cha = tr.stats.channel
            t0: UTCDateTime = tr.stats.starttime
            t1: UTCDateTime = tr.stats.endtime
            start_str = t0.strftime(TIME_FMT)
            end_str = t1.strftime(TIME_FMT)

            # Guardar directamente en la carpeta de salida sin subcarpetas
            out_fname = f"{net}.{sta}..{cha}__{start_str}__{end_str}.mseed"
            out_path = Path(OUTPUT_BASE) / out_fname

            try:
                tr.write(str(out_path), format="MSEED")
                print(f"[OK] {out_path}")
            except Exception as exc:
                print(f"[ERROR] No se pudo escribir {out_path}: {exc}")


if __name__ == "__main__":
    main()

Estandarización del nombre de los mseed NETWORK.STATION..INSTRUMENT__IDENTIFICADOR.mseed

In [ ]:
"""
Une las trazas N, E y Z de cada estación y evento sísmico en un solo archivo MSEED.

- Busca todos los .mseed en INPUT_DIR.
- Agrupa por estación, instrumento (primeras dos letras del canal) y por evento.
- Crea un archivo multicanal por estación, instrumento y evento:
    <NETWORK>.<STATION>..<INSTRUMENT>__<IDENTIFICADOR>.mseed
- Guarda los archivos combinados en OUTPUT_DIR.
"""

import os
import re
from pathlib import Path
from obspy import read, Stream

# -------------------- PARÁMETROS -------------------- #
INPUT_DIR = "F_mseed_eventos_finales"            # Carpeta con las trazas individuales
OUTPUT_DIR = "G_agrupados_por_evento"      # Carpeta destino
# ---------------------------------------------------- #

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Expresión regular para extraer partes del nombre
pattern = re.compile(
    r"^(?P<net>[^.]+)\.(?P<sta>[^.]+)\.\.(?P<cha>[A-Z0-9]+)__"
    r"(?P<start>\d{8}T\d{6}Z)__(?P<end>\d{8}T\d{6}Z)"
)

# Diccionario para agrupar: {(station, instrument, event_id): [archivos]}
groups = {}

for fname in os.listdir(INPUT_DIR):
    if not fname.lower().endswith(".mseed"):
        continue

    match = pattern.match(fname)
    if not match:
        print(f"[WARN] No se pudo interpretar el nombre: {fname}")
        continue

    net = match.group("net")
    sta = match.group("sta")
    cha = match.group("cha")
    instrument = cha[:2]  # Ejemplo: HNE -> HN, EHZ -> EH, etc.
    event_id = f"{match.group('start')}__{match.group('end')}"
    key = (net, sta, instrument, event_id)

    groups.setdefault(key, []).append(fname)

# Procesar cada grupo (estación + instrumento + evento)
for (net, sta, instrument, event_id), files in groups.items():
    st_total = Stream()

    for f in files:
        try:
            tr = read(os.path.join(INPUT_DIR, f))
            st_total += tr
        except Exception as exc:
            print(f"[ERROR] No se pudo leer {f}: {exc}")

    if len(st_total) == 0:
        continue

    out_name = f"{net}.{sta}..{instrument}__{event_id}.mseed"
    out_path = Path(OUTPUT_DIR) / out_name

    try:
        st_total.write(str(out_path), format="MSEED")
        print(f"[OK] {out_name} con {len(st_total)} canales")
    except Exception as exc:
        print(f"[ERROR] No se pudo escribir {out_name}: {exc}")

print("\n Agrupación completada por estación, instrumento y evento.")

ULTIMO RECORTE PARA 6000 MUESTRAS

In [ ]:
from obspy import read, Stream
import numpy as np
import glob
import os

# ===== CONFIGURACIÓN =====
CARPETA_ENTRADA = "G_agrupados_por_evento"   # Cambia por tu carpeta real
CARPETA_SALIDA  = "H_ajustados_6000"       # Cambia por tu carpeta de salida
SAMPLES_OBJETIVO = 6000
# =========================

os.makedirs(CARPETA_SALIDA, exist_ok=True)

def ajustar_y_guardar(mseed_path):
    st = read(mseed_path)

    if len(st) != 3:
        print(f" {os.path.basename(mseed_path)} tiene {len(st)} trazas, se omite.")
        return

    st_ajustado = Stream()

    for tr in st:
        data = tr.data
        n = len(data)

        if n > SAMPLES_OBJETIVO:
            data = data[:SAMPLES_OBJETIVO]  # <- corta del FINAL
        elif n < SAMPLES_OBJETIVO:
            data = np.pad(data, (0, SAMPLES_OBJETIVO - n), 'constant')  # <- rellena al FINAL

        tr.data = data
        st_ajustado.append(tr)

    ruta_salida = os.path.join(CARPETA_SALIDA, os.path.basename(mseed_path))
    st_ajustado.write(ruta_salida, format="MSEED")

    print(f"{os.path.basename(mseed_path)} guardado en salida con 6000 samples por traza.")

for archivo in glob.glob(os.path.join(CARPETA_ENTRADA, "*.mseed")):
    ajustar_y_guardar(archivo)

print("\n Proceso terminado, Todos los archivos en salida tienen 3 canales de 6000 samples.")


CREACION DEL HDF5 Y METADATOS

In [ ]:
import os
import h5py
import json
import numpy as np
from obspy import read
from obspy.core.utcdatetime import UTCDateTime

# ======== CONFIGURACIÓN ========
input_folder = r"E:/TG/BADASI/H_ajustados_6000"      # Carpeta con tus archivos .mseed
output_file = r"E:/TG/BADASI/REDNE_plantilla.hdf5"        # Archivo de salida HDF5
json_stations = r"E:/TG/BADASI/UbiacionEstaciones/estacionesREDNE_data.json"  # Archivo JSON con metadatos de estaciones

# ======== CARGAR JSON DE ESTACIONES ========
with open(json_stations, "r", encoding="utf-8") as f:
    estaciones_data = json.load(f)

# ======== FUNCIÓN PRINCIPAL ========
def convert_mseed_to_hdf5(input_folder, output_file):
    with h5py.File(output_file, "a") as hdf:
        data_group = hdf.require_group("data")

        # Recorrer archivos nuevos de la carpeta
        for file_name in os.listdir(input_folder):
            if not file_name.endswith(".mseed"):
                continue

            file_path = os.path.join(input_folder, file_name)
            st = read(file_path)

            traces = [tr.data.astype(np.float32) for tr in st]
            min_len = min(len(tr) for tr in traces)
            data_matrix = np.vstack([tr[:min_len] for tr in traces]).T

            tr0 = st[0]
            station = tr0.stats.station
            network = tr0.stats.network
            channel_full = tr0.stats.channel
            channel_base = channel_full[:-1] if len(channel_full) > 2 else channel_full

            starttime = tr0.stats.starttime
            start_str = starttime.strftime("%Y%m%d%H%M%S")
            trace_name = f"{station}.{network}_{start_str}_EV"

            # Cargar metadatos desde JSON
            est_info = estaciones_data.get(station, {})
            coords = est_info.get("coords", [None, None, None])
            lat, lon, elev = coords if len(coords) == 3 else (None, None, None)
            network_from_json = est_info.get("network", network)


            utctime = UTCDateTime("2024-05-28T02:54:58.660000Z")
            formatted_time = utctime.datetime.strftime('%Y-%m-%d %H:%M:%S.%f')

            basic_info = {
                "trace_name": trace_name,
                "trace_start_time": formatted_time,
                "trace_channel": channel_base,
                "trace_category": "earthquake_local",
                "station_code": station,
                "station_network_code": network_from_json,
                "station_latitude_deg": lat,
                "station_longitude_deg": lon,
                "station_elevation_m": elev,
            }

            # ======== NO SE BORRAR SI YA EXISTE ========
            if trace_name in data_group:
                continue
            else:
                # Crear dataset solo si es nuevo
                dset = data_group.create_dataset(
                    trace_name,
                    data=data_matrix,
                    dtype=np.float32,
                    compression="gzip"
                )

                # Guardar SOLO los metadatos que realmente existen
                for key, value in basic_info.items():
                    if value is None:
                        continue  # No guardar nada si está vacío

                    if isinstance(value, str):
                        dset.attrs[key] = value
                    else:
                        dset.attrs[key] = np.float32(value)


    print("\n Proceso terminado\n")

# ======== EJECUTAR ========
convert_mseed_to_hdf5(input_folder, output_file)



 Proceso terminado



In [ ]:
# import os
# import shutil

# folders = [
#   r"E:/TG/BADASI/A_REDNE_mseed_downloads",
#   r"E:/TG/BADASI/B_REDNE_mseed_UIS05virtual",
#   r"E:/TG/BADASI/C_Valid_mseed_files",
#   r"E:/TG/BADASI/D_REDNE_mseed_tendencia_media",
#   r"E:/TG/BADASI/E_mseed_eventos_umbrales_finales",
#   r"E:/TG/BADASI/F_mseed_eventos_finales"
#   r"E:/TG/BADASI/G_agrupados_por_evento" 
#   r"E:/TG/BADASI/H_ajustados_6000" 
# ]

# for folder in folders:
#     if os.path.exists(folder):
#         for item in os.listdir(folder):
#             item_path = os.path.join(folder, item)

#             if os.path.isfile(item_path) or os.path.islink(item_path):
#                 os.remove(item_path)  # elimina archivos y enlaces
#             elif os.path.isdir(item_path):
#                 shutil.rmtree(item_path)  # elimina subcarpetas

#         print(f" Contenido eliminado en: {folder}")

# print("\n Limpieza de los directorios completada")
